***GENERATED CODE FOR regressiontestwithdataprepdatecoversion PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***OPERATION FUNCTIONS***

In [ ]:
import pyspark
from dask.dataframe import from_pandas
import json


def DateTransform(df, functionsData, applyOn=[]):
    import dask.dataframe as dd
    import pandas as pd
    import numpy as np

    for functiondata in functionsData:
        functiondata = functiondata

    dfcp = df
    dfcp1 = dfcp
    resulttype = dfcp1.select_dtypes(include=[np.datetime64])
    for existingdatecols in resulttype.columns:
        dfcp[existingdatecols] = dfcp[existingdatecols].apply(
            lambda x: x.replace(tzinfo=None))

    Dateformat1 = 'YYYY-DD-MM'
    Dateformat2 = 'YY/MM/DD'
    Dateformat3 = 'DD/MM/YYYY'
    Dateformat4 = 'M/D/YY'
    NotDateFromat = [Dateformat1, Dateformat2, Dateformat3, Dateformat4]

    if 'inputDateFormat' in functiondata:
        coldetail = functiondata['column']
        for col in coldetail:
            colname = col['columnName']
        inputFormat = str(functiondata['inputDateFormat'])
        functiondata['outputDateFormat']
        functiondata['newColumn']
    if ('newColumnName' in functiondata):
        if inputFormat in NotDateFromat:
            NewDateCol = functiondata['newColumnName']
            dfcp = dfcp.compute()
            dfcp[NewDateCol] = dfcp[colname].apply(
                lambda x: pd.to_datetime(x, format='%d/%m/%Y', utc=False))
            dfcp[NewDateCol] = dfcp[NewDateCol].apply(
                lambda x: x.strftime("%Y-%m-%d"))
            dfcp[NewDateCol] = pd.to_datetime(dfcp[NewDateCol], utc=False)
            dfcp = dd.from_pandas(dfcp, npartitions=2)
            dfcp[NewDateCol] = dfcp[NewDateCol].apply(
                lambda x: x.replace(tzinfo=None))
    elif (('newColumnName' not in functiondata) and (inputFormat in NotDateFromat)):
        dfcp = dfcp.compute()
        dfcp[colname] = dfcp[colname].apply(
            lambda x: pd.to_datetime(x, format='%d/%m/%Y'))
        dfcp[colname] = dfcp[colname].apply(lambda x: x.strftime("%Y-%m-%d"))
        dfcp[colname] = pd.to_datetime(dfcp[colname], utc=False)
        dfcp = dd.from_pandas(dfcp, npartitions=2)

    return dfcp


def runDataCleansing(sparkDf, spark, config):
    configObj = json.loads(config)
    sparkDf.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = from_pandas((sparkDf.toPandas()), npartitions=5)
    functionList = configObj['functionsApplied']
    Data_Cleansing_Methods = {"replaceBy": replaceValues,
                              "formula": calculateFormula,
                              "aggregate": aggregation,
                              "converttostringtype": changeToString,
                              "editname": renameColumns}
    for function in functionList:
        function['functionName']
        df = Data_Cleansing_Methods[function['functionName']](df, function['functionsData'],
                                                              function['applyOn'])
    sparkDf = spark.createDataFrame(df.compute())

    display(sparkDf.limit(2).toPandas())
    return sparkDf


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'AdmitDate', 'transformation_label': 'String Indexer'}], 'feature': 'AdmitDate', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': '2012-01-01 00:00:00', 'max': '2012-06-15 00:00:00', 'missing': '0', 'distinct': '167'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'AdmitDate'}, {'feature_label': 'AdmitDate', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('AdmitDate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'gender', 'transformation_label': 'String Indexer'}], 'feature': 'gender', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'F', 'max': 'M', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'gender'}, {'feature_label': 'gender', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('gender')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'facid', 'transformation_label': 'String Indexer'}], 'feature': 'facid', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'A', 'max': 'E', 'missing': '0', 'distinct': '5'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'facid'}, {'feature_label': 'facid', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('facid')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.model_selection import train_test_split
from tpot import TPOTRegressor
import pyspark


def functionRegression(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTRegressor(verbosity=3, generations=10, max_time_mins=5,
                              n_jobs=-1, random_state=25, population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Error rate of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run regressiontestwithdataprepdatecoversionHooks.ipynb
try:
	#sourcePreExecutionHook()

	hlosddmmyywupk = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/testdata/1717060202184_HLOSddmmyy13.csv', 'filename': 'HLOSddmmyy13.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'FilePath': '/ManualPipelines/HLOSddmmyy13.csv', 'viewFileName': 'HLOSddmmyy13.csv', 'is_header': 'Use Header Line', 'baseType': 'hdfs', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/', 'results_url': 'http://dnm.bfirst.ai:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PERFORMING OPERATIONS***

In [ ]:
#%run regressiontestwithdataprepdatecoversionHooks.ipynb
try:
	#operationPreExecutionHook()

datapreparation = runDataCleansing(hlosddmmyywupk,spark,json.dumps( {"url": "/FileStore/platform/testdata/1717060202184_HLOSddmmyy13.csv", "DataPrepFile": "/FileStore/platform/testdata/1717060202184_HLOSddmmyy13.csv", "data_source": "platfiles", "startListenerOnly": 1, "dateColumnNames": [], "FilePath": "/FileStore/platform/extra/666545d41cee3507cf7db3191717912673/0part.csv", "requestRatio": 3, "totalRows": 1384, "BasicStats": {"missingValues": 0, "numberOfColumns": 26, "numberOfRows": 1384, "duplicateRowCount": 0, "stats": [{"column": "AdmitDate", "alias": "AdmitDate", "generated": 0, "type": "date", "max": "20131013", "min": "201211", "mean": "", "missing": 0, "stddev": "", "outliers": [], "validation": []}, {"column": "lengthofstay", "alias": "lengthofstay", "generated": 0, "type": "numeric", "max": 16, "min": 1, "mean": 3.0498554913294798, "missing": 0, "stddev": 3.05, "outliers": [11, 11, 12, 11, 11, 11, 11, 11, 15, 11, 11, 11, 14, 16, 13, 14, 14, 11, 13, 11, 11, 13, 11, 11, 12, 13, 15, 13, 14, 12, 12, 12, 12, 13, 12, 12, 13, 11, 12, 11, 12, 11, 12, 11, 11], "validation": []}, {"column": "rcount", "alias": "rcount", "generated": 0, "type": "numeric", "max": 5, "min": 0, "mean": 0.8078034682080925, "missing": 0, "stddev": 1.49, "outliers": [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5], "validation": []}, {"column": "gender", "alias": "gender", "generated": 0, "type": "String", "max": "M", "min": "F", "mean": "", "missing": 0, "stddev": "", "outliers": [], "validation": []}, {"column": "dialysisrenalendstage", "alias": "dialysisrenalendstage", "generated": 0, "type": "numeric", "max": 1, "min": 0, "mean": 0.031791907514450865, "missing": 0, "stddev": 0.18, "outliers": [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], "validation": []}, {"column": "asthma", "alias": "asthma", "generated": 0, "type": "numeric", "max": 1, "min": 0, "mean": 0.016618497109826588, "missing": 0, "stddev": 0.13, "outliers": [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], "validation": []}, {"column": "irondef", "alias": "irondef", "generated": 0, "type": "numeric", "max": 1, "min": 0, "mean": 0.06358381502890173, "missing": 0, "stddev": 0.24, "outliers": [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], "validation": []}, {"column": "pneum", "alias": "pneum", "generated": 0, "type": "numeric", "max": 1, "min": 0, "mean": 0.030346820809248554, "missing": 0, "stddev": 0.17, "outliers": [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], "validation": []}, {"column": "substancedependence", "alias": "substancedependence", "generated": 0, "type": "numeric", "max": 1, "min": 0, "mean": 0.036127167630057806, "missing": 0, "stddev": 0.19, "outliers": [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], "validation": []}, {"column": "psychologicaldisordermajor", "alias": "psychologicaldisordermajor", "generated": 0, "type": "numeric", "max": 1, "min": 0, "mean": 0.13439306358381503, "missing": 0, "stddev": 0.34, "outliers": [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], "validation": []}, {"column": "depress", "alias": "depress", "generated": 0, "type": "numeric", "max": 1, "min": 0, "mean": 0.02023121387283237, "missing": 0, "stddev": 0.14, "outliers": [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], "validation": []}, {"column": "psychother", "alias": "psychother", "generated": 0, "type": "numeric", "max": 1, "min": 0, "mean": 0.03901734104046243, "missing": 0, "stddev": 0.19, "outliers": [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], "validation": []}, {"column": "fibrosisandother", "alias": "fibrosisandother", "generated": 0, "type": "numeric", "max": 1, "min": 0, "mean": 0.002890173410404624, "missing": 0, "stddev": 0.05, "outliers": [1, 1, 1, 1], "validation": []}, {"column": "malnutrition", "alias": "malnutrition", "generated": 0, "type": "numeric", "max": 1, "min": 0, "mean": 0.033236994219653176, "missing": 0, "stddev": 0.18, "outliers": [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], "validation": []}, {"column": "hemo", "alias": "hemo", "generated": 0, "type": "numeric", "max": 1, "min": 0, "mean": 0.05346820809248555, "missing": 0, "stddev": 0.23, "outliers": [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], "validation": []}, {"column": "hematocrit", "alias": "hematocrit", "generated": 0, "type": "real", "max": 23.1, "min": 6.6, "mean": 11.9018364643526, "missing": 0, "stddev": 1.83, "outliers": [14.8, 14.5, 9.4, 16.7, 15, 8.3, 8.8, 7.4, 9.05, 23.1, 15, 8.9, 9.1, 9.2, 7.6, 9.3, 9.3, 16.9, 14.5, 14.8, 15, 8.5, 9.4, 14.9, 16, 8.6, 15.4, 18.1, 8.8, 14.4, 17.7, 8.6, 9.2, 8.4, 9.15, 8.9, 9.2, 7, 17.5, 8.9, 9.2, 14.9, 9.35, 9.1, 8.6, 8.8, 14.85, 8, 8, 9.3, 14.5, 8.4, 8, 9.1, 8.9, 6.95, 7.6, 15, 7.8, 15.8, 9.3, 8.7, 14.8, 8, 16.8, 16.2, 8.2, 15.4, 15, 9.1, 9.05, 9.2, 9.4, 18.6, 8.7, 9.4, 8.2, 8.5, 7.9, 9.3, 17, 8.4, 14.7, 14.5, 8.9, 16.9, 8.5, 8.6, 8.2, 18.4, 8, 8.1, 8.9, 7, 14.55, 9, 8, 15.1, 9, 9.1, 15.1, 9.4, 14.5, 8.9, 18.8, 8.7, 7.5, 15, 14.5, 16.6, 9.4, 7.6, 15.3, 9.2, 16.2, 7.6, 9.3, 19.8, 8.9, 14.4, 18.5, 19.4, 8.4, 8.8, 18.7, 6.6, 9.3, 9, 15.7, 8.9, 8.1, 7.95, 14.5, 9.4, 15.4, 7.8, 14.8, 15.3, 8, 8, 6.666666667, 6.95, 15.1, 14.5, 18.9, 9.2, 9.1, 15.6, 9.2, 15.3, 9.1, 8.066666667, 16, 15.2, 17.6, 9.1, 8.6, 9.2, 14.4, 8.1, 17.8, 16.1, 8.4, 14.4, 9, 15.3, 8.6, 9.1, 9.3, 8.6, 6.8, 8.8, 16.9, 15.2, 9, 21.5, 8.7, 9.4, 8.075, 9.1, 9.3, 7.5, 14.8, 14.6, 14.9, 8.5, 15, 9, 15.6, 14.8, 9, 9.4, 7.7, 8.5, 15.9, 17.3, 7.9, 14.6, 7.4, 18.4, 9, 8.2, 7.3, 14.6, 9.3, 18.45, 15.9, 15, 8.1, 15.8, 14.6, 15, 14.9, 8.3, 8.2, 7.8, 16.7, 7.9, 21.1, 14.5, 14.8, 16.1, 14.8, 8.7, 16.1, 17.1, 15.3, 8.4, 14.4, 8.5, 17.7, 15.8], "validation": []}, {"column": "neutrophils", "alias": "neutrophils", "generated": 0, "type": "real", "max": 37.5, "min": 0.7, "mean": 10.176619701343931, "missing": 0, "stddev": 3.87, "outliers": [15.6, 16.6, 4.6, 27, 3.4, 4, 4.5, 20.1, 16.7, 23.4, 15.7, 18.9, 16.9, 19, 19.1, 19.8, 19, 15.2, 18.8, 30.4, 15.35, 1.8, 4.7, 16.5, 1.45, 15.3, 3.1, 16, 4, 2.1, 16, 15.7, 16, 17.1, 4, 0.7, 3.4, 18.5, 16, 20.4, 18.2, 4.8, 17, 15.3, 4, 25.2, 18.8, 16.6, 37.5, 4.8, 17.1, 4.2, 15.6, 3.6, 2.9, 16.8, 4.9, 4.8, 19.4, 16.3, 23, 3.6, 16.4, 16.3, 16.6, 22.9, 4.4, 17.9, 16.9, 15.2, 16.2, 20.1, 16.1, 15.8, 3.3, 4, 3.2, 17.6, 28.8, 3.5, 18, 16.8, 4.3, 4.4, 4.9, 18, 15.3, 3.8, 15.6, 17.1, 16.3, 19, 16.2, 18.1, 24.7, 26.5, 16.9, 16.5, 16.7, 16.35, 4.8, 15.7, 15.3, 15.6, 16.3, 4.6, 19, 15.6, 20.1, 22.2, 4, 18.5, 15.5, 15.9, 29, 15.9, 30.7, 16.6, 15.4, 15.4, 33.43333333, 1.7, 4.2, 17.5, 16.5, 22, 2, 4.8, 20.7, 18.8, 21.6, 16.3, 15.5, 16, 21, 25.2, 18.1, 18.1, 20.3, 23.3, 15.9, 16.7, 30.3, 2.6, 16.2, 18.4, 18.65, 4.9, 32.3, 19.8, 16, 4.9, 16.7, 20.5, 4.4, 4.7, 16.5, 4.3, 15.7, 20.4, 16.8, 0.8, 3.5, 16.8, 21.5, 4.9, 18.8, 15.9, 4.9, 18.3, 3.8, 4.3, 33.2, 4.3, 4.2, 3.8, 19.3, 16.7, 17.7, 18.6, 4.9, 3.3, 16.2, 3.9, 16.6, 3.9, 4.5], "validation": []}, {"column": "sodium", "alias": "sodium", "generated": 0, "type": "real", "max": 147.6734054, "min": 128.4352486, "mean": 137.84580330166185, "missing": 0, "stddev": 2.89, "outliers": [130.897955, 130.9872522, 131.256601, 130.0499267, 144.7350682, 129.0087344, 144.7348389, 130.7803336, 145.6142826, 130.965725, 145.5859386, 146.1556626, 145.4913236, 146.0269477, 131.0263591, 145.2332937, 130.64884, 130.7555452, 131.2138859, 145.3766152, 131.5862113, 129.5012903, 131.6715463, 130.8564015, 144.4758602, 147.3368132, 144.8621759, 144.6312842, 128.4352486, 144.4916274, 129.5006517, 131.4523506, 145.402618, 131.6457329, 145.0623716, 147.6734054, 131.3063375, 130.9484925, 129.1890081, 144.6142898, 129.512507, 144.5546057, 146.0746105, 145.9085343, 144.8114685, 129.8903773], "validation": []}, {"column": "glucose", "alias": "glucose", "generated": 0, "type": "real", "max": 249.0782194, "min": 47.21898217, "mean": 142.32760474276012, "missing": 0, "stddev": 28.47, "outliers": [217.5513409, 67.14852086, 216.8761371, 249.0782194, 219.6736558, 73.40768155, 235.9927636, 70.6148183, 64.63894113, 215.0350009, 71.24874423, 74.50987608, 216.1800955, 216.2341472, 70.74279486, 72.34690242, 66.71718848, 71.25646776, 213.7993875, 47.21898217, 213.5903601, 248.9076823, 72.15010105, 213.0057066], "validation": []}, {"column": "bloodureanitro", "alias": "bloodureanitro", "generated": 0, "type": "real", "max": 145.3333333, "min": 3, "mean": 13.804431599205204, "missing": 0, "stddev": 9.19, "outliers": [11, 15, 14, 18, 8, 5, 7, 10, 7, 10, 41, 29, 5, 64, 16, 4, 6, 17, 4, 19, 9, 9, 19, 6, 11, 4, 6, 5, 5, 10, 14, 18, 10, 22, 5, 11, 17, 6, 7, 13, 18, 17, 19, 54, 11, 11, 13, 10, 30, 15, 16, 28, 8, 10, 13, 20.5, 15, 11.5, 21, 14, 16, 4, 13, 10, 15, 7, 5, 11, 9, 15, 7, 13, 11, 5, 9, 8, 17, 16, 16, 17, 9, 10, 14, 35, 28, 10, 24, 10, 11, 10, 25, 9, 3, 22, 9, 9, 17, 4.25, 5, 7, 10, 25, 9, 11, 18, 13, 11, 15, 40, 14, 6, 61, 16, 7, 19, 14, 44, 77, 27, 25, 53, 6, 5, 42, 25, 36, 7, 10, 11, 9, 23, 8, 5, 11, 11, 14, 18, 5, 24, 9, 7, 15, 11, 7, 10, 10, 6, 21, 20, 9, 13, 7, 15, 13, 6, 19, 75, 56, 6, 10, 6, 7.5, 32, 26, 42, 15, 20, 11, 28, 16, 14, 40, 5, 7, 15, 4, 6, 7, 8, 15, 13, 5, 15, 8, 39, 9, 13, 14, 17, 13, 11, 13, 10, 9, 10, 10, 6, 36, 11, 5, 9, 16, 10, 61, 8, 10, 32, 34, 35, 13, 11, 11, 21, 7, 14, 22, 11, 28, 23, 9, 28, 11, 10, 6, 16, 11, 13, 9, 6, 23, 7, 13, 17, 9, 8, 10, 10, 58, 10, 7, 42, 23, 8, 35, 6, 11, 15, 13, 6, 47, 10, 43, 24, 20, 18, 16, 17, 7, 17, 5, 13, 8, 13, 15, 11, 22, 5, 8, 22, 14, 5, 17, 27, 13, 8, 7, 4, 8, 44, 10, 9, 48, 13, 35, 29, 7, 17, 7, 24, 41, 5, 25.5, 6, 7, 60, 18, 15, 13, 34, 11, 14, 17, 15, 7, 8.5, 4, 4, 13, 15, 10.25, 11, 9, 10, 10, 8, 9, 16, 34, 21, 9, 29, 45, 20, 10, 3, 16, 25, 18, 35, 30, 6, 11, 15, 16, 15, 8, 24, 9, 15, 13, 11, 6, 7, 18, 11, 6, 21, 8, 14, 6, 15, 7, 21, 13, 23, 3, 5, 31, 11, 9, 10, 17, 10, 72, 8, 11, 19, 14, 9, 9, 14, 8, 9, 18, 15, 26, 5, 14, 13, 22, 11, 6, 4, 14, 7, 3, 15, 9, 5, 15, 15, 14, 7, 7, 7.5, 40, 6, 10, 5, 9, 14, 10, 31, 19, 8, 16, 15, 8, 14, 15, 7, 18, 9, 14, 7, 4, 10, 11, 10, 7, 15, 9, 7, 15, 10, 145.3333333, 8, 17, 11, 18, 37, 21, 33, 9, 50, 74, 16, 24, 61, 70, 19, 17, 28.5, 7, 20, 3, 10, 20, 14, 16, 41, 9, 6, 10, 20, 6, 39, 26.5, 21, 7, 16, 4, 19, 18, 3, 19, 33, 17, 9, 11, 17, 9, 19, 11, 9, 6, 9, 26, 4, 25, 9, 30, 14, 11, 13, 11, 11, 15, 13, 5, 83, 21, 6, 8, 10, 26, 9, 18, 32, 14, 30, 16, 10, 24, 7, 13, 9, 14, 28, 6, 4, 6, 39, 18, 10, 13, 16, 35, 14, 18, 6, 9, 16, 6, 11, 53, 11, 9, 74.5, 11, 14, 23, 17, 8, 25, 15, 15, 6, 9, 9, 11, 14, 17, 14, 11, 8, 5, 9, 8, 21, 13.5, 36, 9, 23, 14, 11, 20, 87, 10, 11, 24, 26, 11, 16, 6.5, 23, 7, 13, 6, 17, 13, 9, 38, 10, 8, 10, 10, 15, 3, 10, 10, 6, 16, 8, 17, 22, 26, 17, 5, 14, 16, 18, 16, 11, 24, 10, 25, 13, 14, 4, 7, 13], "validation": []}, {"column": "creatinine", "alias": "creatinine", "generated": 0, "type": "real", "max": 1.817174943, "min": 0.502253806, "mean": 1.1041911003554912, "missing": 0, "stddev": 0.19, "outliers": [1.671152623, 0.647600171, 1.580547458, 1.817174943, 1.579515854, 1.610202243, 1.608949414, 1.759563091, 0.502253806, 0.640800379, 0.628823082, 0.632529977, 0.618242277, 0.585372682, 0.576633974, 0.641164433, 1.592733477, 1.585313601, 1.640736145, 1.721810542, 0.603728088, 1.629944978, 1.613905837, 1.654858411], "validation": []}, {"column": "bmi", "alias": "bmi", "generated": 0, "type": "real", "max": 35.56575783, "min": 23.37224791, "mean": 29.77738727320087, "missing": 0, "stddev": 1.91, "outliers": [25.00999592, 24.1723888, 34.45018493, 24.56990145, 25.22086729, 34.28038553, 24.20511504, 35.40997817, 25.10319269, 24.42077021, 24.9985545, 24.77590915, 25.16083398, 34.69661906, 34.7232434, 24.72968308, 35.56575783, 34.32634137, 35.52976136, 23.37224791, 25.22152151, 25.17006612, 24.1437635, 24.71208143, 25.02874736, 25.17576026, 24.55207325, 25.11550388], "validation": []}, {"column": "pulse", "alias": "pulse", "generated": 0, "type": "numeric", "max": 111, "min": 37, "mean": 73.91835260115607, "missing": 0, "stddev": 10.62, "outliers": [44, 106, 103, 47, 111, 44, 47, 48, 48, 44, 103, 46, 43, 45, 43, 47, 101, 47, 103, 105, 46, 46, 101, 37, 45], "validation": []}, {"column": "respiration", "alias": "respiration", "generated": 0, "type": "real", "max": 8.9, "min": 0.3, "mean": 6.4813041907514455, "missing": 0, "stddev": 0.55, "outliers": [6.2, 6.3, 6.3, 6.8, 5.3, 4.5, 6.2, 5.6, 7.2, 6.3, 6.75, 7.5, 7.3, 7.5, 7.1, 7, 6.6, 5.6, 6.05, 5.9, 7.9, 4.7, 5.4, 6, 8.2, 7.2, 7.5, 7.5, 7.6, 6.9, 6, 6.9, 5.6, 5.1, 6.8, 6.8, 6.4, 6.2, 7, 7.2, 5.3, 7.1, 6.3, 6.7, 5.6, 5.8, 5.8, 6.6, 7.4, 6.1, 6, 7.7, 6.9, 6.3, 7.75, 6.6, 5.4, 6.7, 5.9, 5.25, 7.2, 6.4, 7.8, 6.4, 6.8, 6.2, 7.4, 7.2, 5.3, 7.8, 8.8, 4.2, 7.8, 7.5, 5.7, 5, 6.1, 5.2, 5.2, 4.5, 7.5, 8.1, 4.4, 5, 6.9, 6.05, 7.25, 5.65, 7.2, 6.2, 6.1, 6.2, 6.55, 5.8, 7.6, 5.8, 4.85, 7.15, 6.2, 6.2, 7.4, 7.3, 7, 5.95, 7.2, 6.4, 7.1, 5.8, 7, 7, 6.7, 5.7, 6.8, 7, 7.9, 6.2, 5.2, 7.5, 6, 5.2, 7.9, 6.1, 5.8, 7.5, 5.5, 6.8, 6.4, 8.8, 6.7, 5.7, 6.4, 6.8, 6.6, 6.6, 5.6, 7.5, 5.5, 5.9, 7.1, 7.4, 7.5, 7.1, 7.9, 6.9, 6.45, 6.9, 7.2, 6.4, 5.2, 6.2, 7.3, 6.6, 7, 7.4, 6.2, 7, 6.8, 6.6, 7.1, 6.6, 7.3, 5.9, 5.5, 7.9, 6.7, 5.9, 1.1, 7.7, 7.7, 7.6, 5.7, 5.5, 5.1, 7.1, 6.1, 7, 7.4, 8, 5.6, 5.8, 6.1, 6.05, 5.6, 6.8, 6.1, 6.4, 6.675, 5.8, 5.2, 5.8, 6.8, 6.7, 6.7, 5.3, 5.5, 4.7, 4.9, 7.1, 5.2, 7.7, 6.3, 6.3, 7.3, 6.1, 7, 5.2, 6.4, 6.7, 5.8, 6.6, 6.3, 6.4, 6.4, 6.8, 6.4, 6.2, 7, 6.9, 5.5, 5.7, 5.1, 5.7, 6, 8.3, 8.2, 5.8, 7.5, 6.4, 8, 5.6, 6.4, 7.5, 5.8, 6.3, 5.3, 6.7, 6.4, 6.4, 5.15, 5.9, 6.7, 8.4, 5.4, 4.25, 5.6, 6.7, 5.1, 8.2, 6.7, 5.8, 8.4, 7.4, 5.8, 6.9, 6.6, 5.9, 6.1, 7.1, 6.3, 6.7, 6.4, 6.8, 6.2, 5.6, 6.8, 5.8, 5.7, 5, 5.7, 0.3, 7, 6.3, 7.7, 7.4, 6.1, 7, 7.2, 6.4, 6.4, 5.6, 5.6, 5.5, 5.3, 7, 6.2, 6.1, 6, 6.6, 7.6, 8.9, 8, 6.7, 7, 6.1, 5.6, 6.3, 7.1, 5.6, 8.4, 7, 7.2, 7.1, 7.4, 6.6, 6.3, 6.3, 6.2, 4.3, 6.8, 6.7, 6.7, 6.9, 8.6, 8, 5.7, 7.6, 7.4, 7.6, 6.9, 5.2, 6.8, 5.8, 5.6, 6, 4.85, 6, 5, 7.8, 5.8, 5.4, 6.7, 6.4, 5.8, 7.2, 7, 7.5, 7.5, 1.6, 6.7, 6, 6.3, 7.1, 6, 5.9, 7.1, 5.9, 7.1, 7.45, 7.2, 7.4, 7.8, 6.8, 5.5, 6.8, 7.9, 5.3, 6.4, 7.7, 5.9, 8, 6.1, 6.7, 7.3, 6.7, 6.8, 4.7, 6, 4.9, 6.7, 5.9, 6.4, 7.3, 7.2, 6, 5.8, 6.6, 0.5, 6.2, 6.6, 6.3, 5.6, 6.2], "validation": []}, {"column": "secondarydiagnosisnonicd9", "alias": "secondarydiagnosisnonicd9", "generated": 0, "type": "numeric", "max": 10, "min": 0, "mean": 2.129335260115607, "missing": 0, "stddev": 2.11, "outliers": [10, 6, 10, 10, 7, 10, 8, 7, 6, 7, 8, 7, 6, 8, 10, 7, 9, 7, 7, 7, 7, 6, 6, 6, 6, 6, 9, 9, 6, 6, 9, 9, 9, 7, 6, 6, 8, 7, 8, 6, 9, 6, 6, 10, 10, 6, 6, 6, 10, 10, 6, 9, 9, 8, 7, 6, 7, 8, 8, 10, 8, 7, 7, 9, 7, 7, 7, 7, 9, 7, 6, 10, 9, 6, 8, 10, 9, 6, 10, 10, 6, 7, 8, 6, 6, 7, 10, 6, 7, 9, 6, 8, 8, 9, 9, 8, 7, 9, 6, 10, 9, 6, 10, 6, 6, 8, 6, 6, 6, 6, 6, 6, 6, 6, 10], "validation": []}, {"column": "facid", "alias": "facid", "generated": 0, "type": "String", "max": "E", "min": "A", "mean": "", "missing": 0, "stddev": "", "outliers": [], "validation": []}]}, "predictionPowerScore": [{"AdmitDate": 1, "asthma": 0, "bloodureanitro": 0, "bmi": 0, "creatinine": 0, "depress": 0, "dialysisrenalendstage": 0, "facid": 0, "fibrosisandother": 0, "gender": 0, "glucose": 0, "hematocrit": 0, "hemo": 0, "irondef": 0, "lengthofstay": 0, "malnutrition": 0, "neutrophils": 0, "pneum": 0, "psychologicaldisordermajor": 0, "psychother": 0, "pulse": 0, "rcount": 0, "respiration": 0, "secondarydiagnosisnonicd9": 0, "sodium": 0, "substancedependence": 0}, {"AdmitDate": 0, "asthma": 1, "bloodureanitro": 0, "bmi": 0, "creatinine": 0, "depress": 0, "dialysisrenalendstage": 0, "facid": 0, "fibrosisandother": 0, "gender": 0, "glucose": 0, "hematocrit": 0, "hemo": 0, "irondef": 0, "lengthofstay": 0, "malnutrition": 0, "neutrophils": 0, "pneum": 0, "psychologicaldisordermajor": 0, "psychother": 0, "pulse": 0, "rcount": 0, "respiration": 0, "secondarydiagnosisnonicd9": 0, "sodium": 0, "substancedependence": 0}, {"AdmitDate": 0, "asthma": 0, "bloodureanitro": 1, "bmi": 0, "creatinine": 0, "depress": 0, "dialysisrenalendstage": 0, "facid": 0, "fibrosisandother": 0, "gender": 0, "glucose": 0, "hematocrit": 0, "hemo": 0, "irondef": 0, "lengthofstay": 0, "malnutrition": 0, "neutrophils": 0, "pneum": 0, "psychologicaldisordermajor": 0, "psychother": 0.0019207298, "pulse": 0, "rcount": 0, "respiration": 0, "secondarydiagnosisnonicd9": 0, "sodium": 0, "substancedependence": 0}, {"AdmitDate": 0, "asthma": 0.0001464296, "bloodureanitro": 0, "bmi": 1, "creatinine": 0, "depress": 0, "dialysisrenalendstage": 0, "facid": 0, "fibrosisandother": 0, "gender": 0, "glucose": 0, "hematocrit": 0, "hemo": 0, "irondef": 0, "lengthofstay": 0, "malnutrition": 0, "neutrophils": 0, "pneum": 0.0001354004, "psychologicaldisordermajor": 0, "psychother": 0, "pulse": 0, "rcount": 0, "respiration": 0, "secondarydiagnosisnonicd9": 0, "sodium": 0, "substancedependence": 0}, {"AdmitDate": 0, "asthma": 0, "bloodureanitro": 0, "bmi": 0, "creatinine": 1, "depress": 0, "dialysisrenalendstage": 0, "facid": 0, "fibrosisandother": 0, "gender": 0.0010038396, "glucose": 0, "hematocrit": 0, "hemo": 0, "irondef": 0.0003205832, "lengthofstay": 0, "malnutrition": 0, "neutrophils": 0, "pneum": 0, "psychologicaldisordermajor": 0, "psychother": 0, "pulse": 0, "rcount": 0, "respiration": 0, "secondarydiagnosisnonicd9": 0, "sodium": 0, "substancedependence": 0.0006473855}, {"AdmitDate": 0, "asthma": 0, "bloodureanitro": 0, "bmi": 0, "creatinine": 0, "depress": 1, "dialysisrenalendstage": 0, "facid": 0, "fibrosisandother": 0, "gender": 0, "glucose": 0, "hematocrit": 0, "hemo": 0, "irondef": 0, "lengthofstay": 0, "malnutrition": 0, "neutrophils": 0, "pneum": 0, "psychologicaldisordermajor": 0, "psychother": 0, "pulse": 0, "rcount": 0, "respiration": 0, "secondarydiagnosisnonicd9": 0, "sodium": 0, "substancedependence": 0}, {"AdmitDate": 0, "asthma": 0, "bloodureanitro": 0, "bmi": 0, "creatinine": 0, "depress": 0, "dialysisrenalendstage": 1, "facid": 0, "fibrosisandother": 0, "gender": 0, "glucose": 0, "hematocrit": 0, "hemo": 0, "irondef": 0, "lengthofstay": 0, "malnutrition": 0, "neutrophils": 0, "pneum": 0, "psychologicaldisordermajor": 0, "psychother": 0, "pulse": 0, "rcount": 0, "respiration": 0, "secondarydiagnosisnonicd9": 0, "sodium": 0, "substancedependence": 0}, {"AdmitDate": 0.0439146564, "asthma": 0, "bloodureanitro": 0.0591655776, "bmi": 0, "creatinine": 0.0073919578, "depress": 0, "dialysisrenalendstage": 0, "facid": 1, "fibrosisandother": 0, "gender": 0, "glucose": 0, "hematocrit": 0.0432295498, "hemo": 0, "irondef": 0, "lengthofstay": 0.0743870082, "malnutrition": 0, "neutrophils": 0.0368106014, "pneum": 0, "psychologicaldisordermajor": 0.0834964315, "psychother": 0, "pulse": 0.0110338456, "rcount": 0, "respiration": 0, "secondarydiagnosisnonicd9": 0, "sodium": 0.0186278096, "substancedependence": 0}, {"AdmitDate": 0, "asthma": 0, "bloodureanitro": 0, "bmi": 0, "creatinine": 0, "depress": 0, "dialysisrenalendstage": 0, "facid": 0, "fibrosisandother": 1, "gender": 0, "glucose": 0, "hematocrit": 0, "hemo": 0, "irondef": 0, "lengthofstay": 0, "malnutrition": 0, "neutrophils": 0.25, "pneum": 0, "psychologicaldisordermajor": 0, "psychother": 0, "pulse": 0, "rcount": 0, "respiration": 0, "secondarydiagnosisnonicd9": 0, "sodium": 0, "substancedependence": 0}, {"AdmitDate": 0.0093834585, "asthma": 0, "bloodureanitro": 0.1660667281, "bmi": 0.0703344431, "creatinine": 0.0192453964, "depress": 0, "dialysisrenalendstage": 0, "facid": 0.130357117, "fibrosisandother": 0, "gender": 1, "glucose": 0.0916242438, "hematocrit": 0.2374294175, "hemo": 0, "irondef": 0.0237337548, "lengthofstay": 0.080680667, "malnutrition": 0, "neutrophils": 0.1903588556, "pneum": 0, "psychologicaldisordermajor": 0.0997072254, "psychother": 0, "pulse": 0.0349795065, "rcount": 0.0003807745, "respiration": 0.0662714877, "secondarydiagnosisnonicd9": 0, "sodium": 0.0246967313, "substancedependence": 0}, {"AdmitDate": 0, "asthma": 0, "bloodureanitro": 0, "bmi": 0, "creatinine": 0, "depress": 0, "dialysisrenalendstage": 0, "facid": 0.0011453999, "fibrosisandother": 0, "gender": 0, "glucose": 1, "hematocrit": 0, "hemo": 0, "irondef": 0.0001077251, "lengthofstay": 0, "malnutrition": 0, "neutrophils": 0, "pneum": 0, "psychologicaldisordermajor": 0, "psychother": 0, "pulse": 0, "rcount": 0, "respiration": 0, "secondarydiagnosisnonicd9": 0, "sodium": 0, "substancedependence": 0}, {"AdmitDate": 0, "asthma": 0, "bloodureanitro": 0, "bmi": 0, "creatinine": 0, "depress": 0, "dialysisrenalendstage": 0.0101781714, "facid": 0, "fibrosisandother": 0.0037828062, "gender": 0, "glucose": 0, "hematocrit": 1, "hemo": 0.0518295106, "irondef": 0, "lengthofstay": 0, "malnutrition": 0, "neutrophils": 0, "pneum": 0, "psychologicaldisordermajor": 0, "psychother": 0.0175388092, "pulse": 0, "rcount": 0, "respiration": 0, "secondarydiagnosisnonicd9": 0, "sodium": 0, "substancedependence": 0.0002749565}, {"AdmitDate": 0, "asthma": 0, "bloodureanitro": 0, "bmi": 0, "creatinine": 0, "depress": 0, "dialysisrenalendstage": 0, "facid": 0, "fibrosisandother": 0, "gender": 0, "glucose": 0, "hematocrit": 0, "hemo": 1, "irondef": 0, "lengthofstay": 0, "malnutrition": 0, "neutrophils": 0, "pneum": 0, "psychologicaldisordermajor": 0, "psychother": 0, "pulse": 0, "rcount": 0, "respiration": 0, "secondarydiagnosisnonicd9": 0, "sodium": 0, "substancedependence": 0}, {"AdmitDate": 0, "asthma": 0, "bloodureanitro": 0, "bmi": 0, "creatinine": 0, "depress": 0, "dialysisrenalendstage": 0, "facid": 0, "fibrosisandother": 0, "gender": 0, "glucose": 0, "hematocrit": 0, "hemo": 0, "irondef": 1, "lengthofstay": 0, "malnutrition": 0, "neutrophils": 0, "pneum": 0, "psychologicaldisordermajor": 0, "psychother": 0, "pulse": 0, "rcount": 0, "respiration": 0, "secondarydiagnosisnonicd9": 0, "sodium": 0, "substancedependence": 0}, {"AdmitDate": 0.5858301022, "asthma": 0, "bloodureanitro": 0, "bmi": 0, "creatinine": 0, "depress": 0, "dialysisrenalendstage": 0, "facid": 0.104596149, "fibrosisandother": 0, "gender": 0, "glucose": 0, "hematocrit": 0, "hemo": 0, "irondef": 0, "lengthofstay": 1, "malnutrition": 0, "neutrophils": 0, "pneum": 0, "psychologicaldisordermajor": 0, "psychother": 0, "pulse": 0, "rcount": 0.4485748734, "respiration": 0, "secondarydiagnosisnonicd9": 0, "sodium": 0, "substancedependence": 0}, {"AdmitDate": 0, "asthma": 0, "bloodureanitro": 0, "bmi": 0, "creatinine": 0, "depress": 0, "dialysisrenalendstage": 0, "facid": 0, "fibrosisandother": 0, "gender": 0, "glucose": 0, "hematocrit": 0, "hemo": 0, "irondef": 0, "lengthofstay": 0, "malnutrition": 1, "neutrophils": 0, "pneum": 0, "psychologicaldisordermajor": 0, "psychother": 0, "pulse": 0, "rcount": 0, "respiration": 0, "secondarydiagnosisnonicd9": 0, "sodium": 0, "substancedependence": 0}, {"AdmitDate": 0, "asthma": 0, "bloodureanitro": 0, "bmi": 0, "creatinine": 0, "depress": 0, "dialysisrenalendstage": 0, "facid": 0, "fibrosisandother": 0, "gender": 0, "glucose": 0, "hematocrit": 0, "hemo": 0, "irondef": 0, "lengthofstay": 0, "malnutrition": 0, "neutrophils": 1, "pneum": 0, "psychologicaldisordermajor": 0, "psychother": 0, "pulse": 0, "rcount": 0, "respiration": 0, "secondarydiagnosisnonicd9": 0, "sodium": 0, "substancedependence": 0}, {"AdmitDate": 0, "asthma": 0, "bloodureanitro": 0, "bmi": 0, "creatinine": 0, "depress": 0, "dialysisrenalendstage": 0, "facid": 0, "fibrosisandother": 0, "gender": 0, "glucose": 0, "hematocrit": 0, "hemo": 0, "irondef": 0, "lengthofstay": 0, "malnutrition": 0, "neutrophils": 0, "pneum": 1, "psychologicaldisordermajor": 0, "psychother": 0, "pulse": 0, "rcount": 0, "respiration": 0, "secondarydiagnosisnonicd9": 0, "sodium": 0, "substancedependence": 0}, {"AdmitDate": 0, "asthma": 0, "bloodureanitro": 0, "bmi": 0, "creatinine": 0, "depress": 0, "dialysisrenalendstage": 0, "facid": 0.2682567704, "fibrosisandother": 0, "gender": 0, "glucose": 0, "hematocrit": 0, "hemo": 0, "irondef": 0, "lengthofstay": 0, "malnutrition": 0, "neutrophils": 0, "pneum": 0, "psychologicaldisordermajor": 1, "psychother": 0, "pulse": 0, "rcount": 0, "respiration": 0, "secondarydiagnosisnonicd9": 0, "sodium": 0, "substancedependence": 0}, {"AdmitDate": 0, "asthma": 0, "bloodureanitro": 0, "bmi": 0, "creatinine": 0, "depress": 0, "dialysisrenalendstage": 0, "facid": 0, "fibrosisandother": 0, "gender": 0, "glucose": 0, "hematocrit": 0, "hemo": 0, "irondef": 0, "lengthofstay": 0, "malnutrition": 0, "neutrophils": 0, "pneum": 0, "psychologicaldisordermajor": 0, "psychother": 1, "pulse": 0, "rcount": 0, "respiration": 0, "secondarydiagnosisnonicd9": 0, "sodium": 0, "substancedependence": 0}, {"AdmitDate": 0, "asthma": 0, "bloodureanitro": 0, "bmi": 0, "creatinine": 0, "depress": 0, "dialysisrenalendstage": 0, "facid": 0, "fibrosisandother": 0, "gender": 0, "glucose": 0, "hematocrit": 0, "hemo": 0, "irondef": 0, "lengthofstay": 0, "malnutrition": 0, "neutrophils": 0, "pneum": 0, "psychologicaldisordermajor": 0, "psychother": 0, "pulse": 1, "rcount": 0, "respiration": 0, "secondarydiagnosisnonicd9": 0, "sodium": 0, "substancedependence": 0}, {"AdmitDate": 0.0991353608, "asthma": 0, "bloodureanitro": 0, "bmi": 0, "creatinine": 0, "depress": 0, "dialysisrenalendstage": 0, "facid": 0, "fibrosisandother": 0, "gender": 0, "glucose": 0, "hematocrit": 0, "hemo": 0, "irondef": 0, "lengthofstay": 0.5030865255, "malnutrition": 0, "neutrophils": 0, "pneum": 0, "psychologicaldisordermajor": 0, "psychother": 0, "pulse": 0, "rcount": 1, "respiration": 0, "secondarydiagnosisnonicd9": 0, "sodium": 0, "substancedependence": 0}, {"AdmitDate": 0, "asthma": 0, "bloodureanitro": 0, "bmi": 0, "creatinine": 0, "depress": 0, "dialysisrenalendstage": 0, "facid": 0, "fibrosisandother": 0, "gender": 0, "glucose": 0, "hematocrit": 0, "hemo": 0, "irondef": 0, "lengthofstay": 0, "malnutrition": 0, "neutrophils": 0, "pneum": 0, "psychologicaldisordermajor": 0, "psychother": 0, "pulse": 0, "rcount": 0, "respiration": 1, "secondarydiagnosisnonicd9": 0, "sodium": 0, "substancedependence": 0}, {"AdmitDate": 0, "asthma": 0, "bloodureanitro": 0, "bmi": 0, "creatinine": 0, "depress": 0, "dialysisrenalendstage": 0, "facid": 0, "fibrosisandother": 0, "gender": 0, "glucose": 0, "hematocrit": 0, "hemo": 0, "irondef": 0, "lengthofstay": 0, "malnutrition": 0, "neutrophils": 0, "pneum": 0, "psychologicaldisordermajor": 0, "psychother": 0, "pulse": 0, "rcount": 0, "respiration": 0, "secondarydiagnosisnonicd9": 1, "sodium": 0, "substancedependence": 0}, {"AdmitDate": 0, "asthma": 0, "bloodureanitro": 0, "bmi": 0, "creatinine": 0, "depress": 0, "dialysisrenalendstage": 0, "facid": 0, "fibrosisandother": 0, "gender": 0.000150767, "glucose": 0, "hematocrit": 0, "hemo": 0, "irondef": 0, "lengthofstay": 0, "malnutrition": 0, "neutrophils": 0, "pneum": 0, "psychologicaldisordermajor": 0, "psychother": 0, "pulse": 0, "rcount": 0, "respiration": 0, "secondarydiagnosisnonicd9": 0, "sodium": 1, "substancedependence": 0}, {"AdmitDate": 0, "asthma": 0, "bloodureanitro": 0, "bmi": 0, "creatinine": 0, "depress": 0, "dialysisrenalendstage": 0, "facid": 0, "fibrosisandother": 0, "gender": 0, "glucose": 0, "hematocrit": 0, "hemo": 0, "irondef": 0, "lengthofstay": 0, "malnutrition": 0, "neutrophils": 0, "pneum": 0, "psychologicaldisordermajor": 0, "psychother": 0, "pulse": 0, "rcount": 0, "respiration": 0, "secondarydiagnosisnonicd9": 0, "sodium": 0, "substancedependence": 1}], "HasBasicStats": 1, "functionsApplied": [{"functionName": "transformDate", "applyOn": [{"columnName": "AdmitDate", "type": "String", "min": "1/1/2012", "max": "9/9/2013", "mean": "-"}], "functionsData": [{"column": [{"columnName": "AdmitDate", "type": "String", "min": "1/1/2012", "max": "9/9/2013", "mean": "-"}], "inputDateFormat": "DD/MM/YYYY", "outputDateFormat": "YYYY-MM-DD", "newColumn": "False"}]}], "functionChanges": [{"columnName": "AdmitDate", "functionName": "transformDate", "Type": "String", "Parameters": [{"column": [{"columnName": "AdmitDate", "type": "String", "min": "1/1/2012", "max": "9/9/2013", "mean": "-"}], "inputDateFormat": "DD/MM/YYYY", "outputDateFormat": "YYYY-MM-DD", "newColumn": "False"}]}], "fileheader": [{"field": "AdmitDate", "alias": "AdmitDate", "generated": 0, "position": 1, "type": "date"}, {"field": "lengthofstay", "alias": "lengthofstay", "generated": 0, "position": 2, "type": "numeric"}, {"field": "rcount", "alias": "rcount", "generated": 0, "position": 3, "type": "numeric"}, {"field": "gender", "alias": "gender", "generated": 0, "position": 4, "type": "String"}, {"field": "dialysisrenalendstage", "alias": "dialysisrenalendstage", "generated": 0, "position": 5, "type": "numeric"}, {"field": "asthma", "alias": "asthma", "generated": 0, "position": 6, "type": "numeric"}, {"field": "irondef", "alias": "irondef", "generated": 0, "position": 7, "type": "numeric"}, {"field": "pneum", "alias": "pneum", "generated": 0, "position": 8, "type": "numeric"}, {"field": "substancedependence", "alias": "substancedependence", "generated": 0, "position": 9, "type": "numeric"}, {"field": "psychologicaldisordermajor", "alias": "psychologicaldisordermajor", "generated": 0, "position": 10, "type": "numeric"}, {"field": "depress", "alias": "depress", "generated": 0, "position": 11, "type": "numeric"}, {"field": "psychother", "alias": "psychother", "generated": 0, "position": 12, "type": "numeric"}, {"field": "fibrosisandother", "alias": "fibrosisandother", "generated": 0, "position": 13, "type": "numeric"}, {"field": "malnutrition", "alias": "malnutrition", "generated": 0, "position": 14, "type": "numeric"}, {"field": "hemo", "alias": "hemo", "generated": 0, "position": 15, "type": "numeric"}, {"field": "hematocrit", "alias": "hematocrit", "generated": 0, "position": 16, "type": "real"}, {"field": "neutrophils", "alias": "neutrophils", "generated": 0, "position": 17, "type": "real"}, {"field": "sodium", "alias": "sodium", "generated": 0, "position": 18, "type": "real"}, {"field": "glucose", "alias": "glucose", "generated": 0, "position": 19, "type": "real"}, {"field": "bloodureanitro", "alias": "bloodureanitro", "generated": 0, "position": 20, "type": "real"}, {"field": "creatinine", "alias": "creatinine", "generated": 0, "position": 21, "type": "real"}, {"field": "bmi", "alias": "bmi", "generated": 0, "position": 22, "type": "real"}, {"field": "pulse", "alias": "pulse", "generated": 0, "position": 23, "type": "numeric"}, {"field": "respiration", "alias": "respiration", "generated": 0, "position": 24, "type": "real"}, {"field": "secondarydiagnosisnonicd9", "alias": "secondarydiagnosisnonicd9", "generated": 0, "position": 25, "type": "numeric"}, {"field": "facid", "alias": "facid", "generated": 0, "position": 26, "type": "String"}]}))
	#operationPostExecutionHook(datapreparation)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run regressiontestwithdataprepdatecoversionHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(datapreparation,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "AdmitDate", "transformation_label": "String Indexer"}], "feature": "AdmitDate", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "2012-01-01 00:00:00", "max": "2012-06-15 00:00:00", "missing": "0", "distinct": "167"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "AdmitDate"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "lengthofstay", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2.68", "stddev": "3.11", "min": "1", "max": "16", "missing": "0"}, "updatedLabel": "lengthofstay"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "rcount", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.67", "stddev": "1.43", "min": "0", "max": "5", "missing": "0"}, "updatedLabel": "rcount"}, {"transformationsData": [{"feature_label": "gender", "transformation_label": "String Indexer"}], "feature": "gender", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "F", "max": "M", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "gender"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "dialysisrenalendstage", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.03", "stddev": "0.16", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "dialysisrenalendstage"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "asthma", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.02", "stddev": "0.13", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "asthma"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "irondef", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.05", "stddev": "0.23", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "irondef"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "pneum", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.02", "stddev": "0.15", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "pneum"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "substancedependence", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.03", "stddev": "0.17", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "substancedependence"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "psychologicaldisordermajor", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.1", "stddev": "0.29", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "psychologicaldisordermajo..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "depress", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.02", "stddev": "0.13", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "depress"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "psychother", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.03", "stddev": "0.17", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "psychother"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "fibrosisandother", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.01", "stddev": "0.08", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "fibrosisandother"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "malnutrition", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.03", "stddev": "0.17", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "malnutrition"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "hemo", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.04", "stddev": "0.2", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "hemo"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "hematocrit", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "11.84", "stddev": "1.72", "min": "6.95", "max": "23.1", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "hematocrit"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "neutrophils", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "10.15", "stddev": "3.67", "min": "0.7", "max": "37.5", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "neutrophils"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "sodium", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "137.76", "stddev": "2.89", "min": "129.0087344", "max": "146.1556626", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "sodium"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "glucose", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "142.07", "stddev": "26.75", "min": "67.14852086", "max": "249.0782194", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "glucose"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "bloodureanitro", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "13.38", "stddev": "7.96", "min": "3.0", "max": "77.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "bloodureanitro"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "creatinine", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1.11", "stddev": "0.18", "min": "0.647600171", "max": "1.817174943", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "creatinine"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "bmi", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "29.67", "stddev": "1.93", "min": "24.1723888", "max": "35.40997817", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "bmi"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "pulse", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "73.86", "stddev": "10.99", "min": "43", "max": "111", "missing": "0"}, "updatedLabel": "pulse"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "respiration", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "6.49", "stddev": "0.43", "min": "4.2", "max": "8.8", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "respiration"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "secondarydiagnosisnonicd9", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2.21", "stddev": "2.18", "min": "0", "max": "10", "missing": "0"}, "updatedLabel": "secondarydiagnosisnonicd9"}, {"transformationsData": [{"feature_label": "facid", "transformation_label": "String Indexer"}], "feature": "facid", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "A", "max": "E", "missing": "0", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "facid"}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run regressiontestwithdataprepdatecoversionHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionRegression(autofe, ["rcount", "dialysisrenalendstage", "asthma", "irondef", "pneum", "substancedependence", "psychologicaldisordermajor", "depress", "psychother", "fibrosisandother", "malnutrition", "hemo", "hematocrit", "neutrophils", "sodium", "glucose", "bloodureanitro", "creatinine", "bmi", "pulse", "respiration", "secondarydiagnosisnonicd9", "AdmitDate_stringindexer", "gender_stringindexer", "facid_stringindexer"], "lengthofstay")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML ['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    df = df[columnNames]
    R2 = np.round(sklearn.metrics.r2_score(y_test, y_predicted), 1)
    Mean_Squared_Error = np.round(sklearn.metrics.mean_squared_error(y_test, y_predicted), 1)
    Mean_Absolute_Error = np.round(sklearn.metrics.mean_absolute_error(y_test, y_predicted), 1)
    display(" R2 score of Prediction on test data    : %s"%R2)
    display(" Mean Squared Error of Prediction on test data    : %s"%Mean_Squared_Error)
    display(" Mean Absolute Error of Prediction on test data   : %s"%Mean_Absolute_Error)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

